<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/Multileyer_Perceptron_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multilayer-Perceptron con tensorflow y keras v2 (modularizando la creación de la red)
En este nuevo notebook vamos a crear una calse que contendra la creación de la arquitectura de nua red neuronal con TF y keras es una version actual del algoritmo descrito en [multilayer v1](https://github.com/TottiPuc/Machine_learning/blob/master/Multileyer_Perceptron_v1.ipynb).

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import tensorflow as tf

Creamos los hiperparametros de la red y los datos sinteticos a ser entrenados

In [0]:
size = 200000
num_empochs = 10
learning_rate = 0.001
#creamos 200000 datos aleatorios enteros entre 0 y 100
x1 = np.random.randint(0,100, size)
x2 = np.random.randint(0,100, size)
x_train = np.dstack((x1,x2))[0] # concatenamos x1 y x2 y tomamos la primera capa de profundidad

Creamos una funcion Y que dependa de x para generas nuestros datos historicos

In [0]:
y_train = 3*(x1**(1/2))+2*(x2**2)
print( "estos son los valores de entrenamiento para 'x' y 'y' y sus respectivas dimensiones ")
print("valores de x {}, con dimension {}".format(x_train,x_train.shape))
print("valores de y {}, con dimension {}".format(y_train,y_train.shape))

estos son los valores de entrenamiento para 'x' y 'y' y sus respectivas dimensiones 
valores de x [[39 92]
 [ 2 58]
 [47 99]
 ...
 [54 61]
 [73 76]
 [66 12]], con dimension (200000, 2)
valores de y [16946.734994    6732.24264069 19622.5669638  ...  7464.04540769
 11577.63201124   312.37211521], con dimension (200000,)


Creamos el modelo con la herramienta keras y su clase sequential que nos permite adicionar capa tras capa dependiendo de lo que queramos contruir.
En esta ocadión modularizamos la arquitectura de la red a través de una clase llamada modelo.

In [0]:
class Modelo(tf.keras.Model):
  #iniciamos el constructor
  def __init__ (self):
    super(Modelo,self).__init__() # iniciamos el constructor de la super clase en este caso la claase modelo de keras
    #creamos nuestra arquitectura que se iniciara al arrancar una instancia
    self.entrada = tf.keras.layers.Dense(64, input_shape=(2,), activation='sigmoid')
    self.oculta = tf.keras.layers.Dense(128, activation='relu')
    self.salida = tf.keras.layers.Dense(2)
    self.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate), loss= tf.keras.losses.MSE) # en este caso hemos uasdo otra forma de optimizar con RMSprop

  def call(self,inputs):
    x = self.entrada(inputs)
    x = self.oculta(x)
    x = self.salida(x)
    return x

Creamos una instancia de nuestra estructura ahora modularizada

In [0]:
modelo = Modelo()

Entrenamos nuestro modelo ya instanciado

In [0]:
modelo.fit(x = x_train, y = y_train, epochs=num_empochs)

Epoch 1/10
6250/6250 [==============================] - 7s 1ms/step - loss: 15952078.0000
Epoch 2/10
6250/6250 [==============================] - 7s 1ms/step - loss: 125017.1797
Epoch 3/10
6250/6250 [==============================] - 7s 1ms/step - loss: 112065.9375
Epoch 4/10
6250/6250 [==============================] - 7s 1ms/step - loss: 104763.5781
Epoch 5/10
6250/6250 [==============================] - 7s 1ms/step - loss: 100798.8125
Epoch 6/10
6250/6250 [==============================] - 7s 1ms/step - loss: 96513.5938
Epoch 7/10
6250/6250 [==============================] - 7s 1ms/step - loss: 92529.1016
Epoch 8/10
6250/6250 [==============================] - 7s 1ms/step - loss: 89657.2656
Epoch 9/10
6250/6250 [==============================] - 7s 1ms/step - loss: 86446.2109
Epoch 10/10
6250/6250 [==============================] - 7s 1ms/step - loss: 82228.9766


Resumen de nuestra estructura

In [0]:
print("Resumen del modelo ")
modelo.summary()

Resumen del modelo 
Model: "modelo"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  192       
_________________________________________________________________
dense_1 (Dense)              multiple                  8320      
_________________________________________________________________
dense_2 (Dense)              multiple                  258       
Total params: 8,770
Trainable params: 8,770
Non-trainable params: 0
_________________________________________________________________


Evaluamos el rendimiento del entrenamiento

In [0]:
score_train = modelo.evaluate(x_train,y_train, verbose=0)
print("error final del entrenamiento {:.0f}".format(score_train))

error final del entrenamiento 7913


#Evaluando el modelo
Generamos nuevos datos sinteticos para evaluar el modelo ( tienen que estar en el mismo rango entre 0 y 100 ya que el entrenamiento fue realizado con este rango de datos)

In [0]:
x3= np.array([100, 9, 62, 79, 94, 91, 71, 41])
x4= np.array([65, 39, 40, 44, 77, 42, 36, 74])
x_test = np.dstack((x3, x4))[0]

y_test = 3*(x3**(1/2)) + 2*(x4**2)

Realizamos predicciones y evaluamos el rendimiento del test

In [0]:
y_pred = modelo.predict(x_test)
score_test = modelo.evaluate(x_test,y_test, verbose=0)
print("El error fina en test: {:.0f}".format(score_test))

El error fina en test: 1116


Realizamos una verificaión sobre el proceso realizado

In [0]:
for i in range(5):
	print ('''Entrada(x): ({}, {}), Saida(y): ({:.0f}), Previsión del Modelo(y_pred): ({:.0f})'''.format(x1[i], x2[i], y_test[i], y_pred[i][0]))

print("\n")

Entrada(x): (39, 92), Saida(y): (8480), Previsión del Modelo(y_pred): (8421)
Entrada(x): (2, 58), Saida(y): (3051), Previsión del Modelo(y_pred): (3099)
Entrada(x): (47, 99), Saida(y): (3224), Previsión del Modelo(y_pred): (3247)
Entrada(x): (6, 80), Saida(y): (3899), Previsión del Modelo(y_pred): (3934)
Entrada(x): (0, 34), Saida(y): (11887), Previsión del Modelo(y_pred): (11888)


